In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 파일 경로
file_path = '../DailyDoodle_ML/teset.xlsx'  # 절대 경로 사용

# 엑셀 파일 읽기
data = pd.read_excel(file_path, engine='openpyxl')

# 텍스트 데이터와 감정 레이블 추출
text_data = data['사람문장1'].dropna()  # 결측값 제거
emotion_labels = data['감정_대분류'][text_data.index]

# 감정 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(emotion_labels)

# 텍스트 데이터 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(text_data).toarray()
y = encoded_labels

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습 및 교차 검증
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

grid_search = GridSearchCV(LogisticRegression(multi_class='ovr', max_iter=1000), param_grid, cv=5, verbose=1)
grid_search.fit(X_train, y_train)

# 최적의 모델
best_model = grid_search.best_estimator_

# 테스트 데이터에 대한 예측
y_pred = best_model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)

# 사용자 입력 텍스트에 대한 감정 분석
def analyze_sentiment(input_text):
    input_vectorized = vectorizer.transform([input_text])
    predicted_emotion_index = best_model.predict(input_vectorized)
    predicted_emotion = label_encoder.inverse_transform(predicted_emotion_index)
    return predicted_emotion[0]

# 사용자 입력 받기 및 감정 분석
def run_sentiment_analysis():
    while True:
        user_input = input("감정을 분석할 문장을 입력하세요 (종료하려면 '종료'를 입력하세요): ")
        if user_input.lower() == '종료':
            print("프로그램을 종료합니다.")
            break
        predicted_emotion = analyze_sentiment(user_input)
        print(f"예측된 감정: {predicted_emotion}")

# 감정 분석 실행
run_sentiment_analysis()


Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be rem

Accuracy: 0.5040674026728647
Classification Report:
               precision    recall  f1-score   support

          기쁨       0.71      0.72      0.71      1206
          당황       0.49      0.43      0.46      1718
          분노       0.47      0.47      0.47      1831
          불안       0.50      0.56      0.52      1891
          상처       0.41      0.44      0.42      1837
          슬픔       0.53      0.47      0.50      1843

    accuracy                           0.50     10326
   macro avg       0.52      0.52      0.52     10326
weighted avg       0.51      0.50      0.50     10326

예측된 감정: 기쁨
예측된 감정: 분노
예측된 감정: 상처
예측된 감정: 상처
예측된 감정: 상처
예측된 감정: 상처
프로그램을 종료합니다.
